In [5]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.spatial.distance import cdist
 
%matplotlib inline

In [4]:
meal = pd.read_table("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/mfp/data.tsv")
meal.head()

,meal_id,user_id,date,meal_sequence,food_ids
0,1,1,2014-09-14,1,"1,2,3,4,5,6"
1,2,1,2014-09-15,1,"1,6"
2,3,1,2014-09-16,1,"8,1,9,6,7"
3,4,1,2014-09-17,1,"1,6,9,10,11,12"
4,5,1,2014-09-18,1,"1,5,13,6,9"


In [5]:
meal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919024 entries, 0 to 1919023
Data columns (total 5 columns):
meal_id          int64
user_id          int64
date             object
meal_sequence    int64
food_ids         object
dtypes: int64(3), object(2)
memory usage: 73.2+ MB


In [6]:
meal.tail()

,meal_id,user_id,date,meal_sequence,food_ids
1919019,1919020,9896,2015-03-14,3,"2288,16"
1919020,1919021,9896,2015-03-15,1,5473
1919021,1919022,9896,2015-03-15,2,563
1919022,1919023,9896,2015-03-15,3,"1523,44"
1919023,1919024,9896,2015-03-15,4,1446


In [7]:
foods = pd.read_table("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/mfp/items.tsv")
foods.head()

,id,item
0,1,egg_dairy__dairy_product__milk
1,2,"fruit__tropical__banana,nut_seed__nut__nut,sna..."
2,3,"dessert__cookies__cooky,egg_dairy__dairy_produ..."
3,4,"condiment__condiment__peanut_butter,dessert__c..."
4,5,"dessert__confectionery__caramel,snack__snack__..."


In [8]:
foods.tail()

,id,item
71710,71711,"condiment__condiment__ketchup,staple__wheat__b..."
71711,71712,"beverage__coffee__coffee,egg_dairy__dairy_prod..."
71712,71713,"meat__shellfish__shrimp,side_dish__fries__fry,..."
71713,71714,"meat__meat__meat,side_dish__soup__soup,staple_..."
71714,71715,"egg_dairy__dairy_product__cheddar,egg_dairy__e..."


In [17]:
json_foods = pd.read_json("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/food_nutrients/foods.json")
json_foods.head()

,name,usda_id,fat,calories,proteins,carbohydrates,serving,nutrients,fibers
0,Beef liver,13326.0,53000,1.91,291000,51000,100.0,"{'Calcium': 60, 'Iron': 65, 'Potassium': 3520,...",NaN
1,Chicken liver,5027.0,65000,1.67,245000,9000,100.0,"{'Calcium': 110, 'Iron': 116, 'Potassium': 263...",NaN
2,Cod liver oil,4589.0,1000000,9.02,0,0,100.0,"{'Vitamin A': 300, 'Iodine': 4}",NaN
3,Tuna,15117.0,81000,1.86,265000,0,150.0,"{'Calcium': 40, 'Iron': 7, 'Potassium': 3330, ...",NaN
4,Goat cheese,1157.0,356000,4.52,305000,22000,50.0,"{'Calcium': 8950, 'Iron': 19, 'Potassium': 480...",NaN


In [4]:
json_nutrients = pd.read_json("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/food_nutrients/nutrients.json")
json_nutrients.head()

,name,rda,wiki,required,type,tui
0,Vitamin A,900.0,https://en.wikipedia.org/wiki/Vitamin_A,True,Vitamins,3000.0
1,Vitamin B1,1200.0,https://en.wikipedia.org/wiki/Thiamine,True,Vitamins,NaN
2,Vitamin B2,1300.0,https://en.wikipedia.org/wiki/Riboflavin,True,Vitamins,NaN
3,Vitamin B3,16000.0,https://en.wikipedia.org/wiki/Vitamin_B3_complex,True,Vitamins,35000.0
4,Vitamin B5,5000.0,https://en.wikipedia.org/wiki/Pantothenic_acid,True,Vitamins,1200000.0


In [13]:
food_diet = pd.read_excel("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/myfoodapediadata/Food_Display_Table.xlsx")
food_diet.tail()

,Food_Code,Display_Name,Portion_Default,Portion_Amount,Portion_Display_Name,Factor,Increment,Multiplier,Grains,Whole_Grains,...,Milk,Meats,Soy,Drybeans_Peas,Oils,Solid_Fats,Added_Sugars,Alcohol,Calories,Saturated_Fats
2009,57301500,"Kashi, puffed",1,1.0,cup,1.0,0.25,0.25,0.84672,0.84672,...,0.0,0.0,0.0,0.0,0.00005,0.0,0.00000,0.0,87.60,0.0792
2010,57303100,Kix cereal,1,1.0,cup,1.0,0.25,0.25,0.67804,0.23782,...,0.0,0.0,0.0,0.0,0.00000,0.0,8.93255,0.0,82.94,0.1100
2011,57303100,Kix cereal,2,1.0,single serving box,1.0,0.50,0.50,0.55476,0.19458,...,0.0,0.0,0.0,0.0,0.00000,0.0,7.30845,0.0,67.86,0.0900
2012,57304100,Life cereal (plain & cinnamon),1,1.0,cup,1.0,0.25,0.25,1.24652,0.80124,...,0.0,0.0,0.0,0.0,0.00000,0.0,32.09710,0.0,165.00,0.3564
2013,57305100,Lucky Charms cereal,1,1.0,cup,1.0,0.25,0.25,0.58290,0.50790,...,0.0,0.0,0.0,0.0,0.00287,0.0,49.57145,0.0,114.00,0.2400


In [15]:
food_needing_condiments = pd.read_excel("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/myfoodapediadata/Foods_Needing_Condiments_Table.xlsx")
food_needing_condiments.tail()

,Survey_Food_Code,display_name,cond_1_code,cond_1_name,cond_2_code,cond_2_name,cond_3_code,cond_3_name,cond_4_code,cond_4_name,cond_5_code,cond_5_name
284,77300000,Wrap sandwich (vegetables only),12310100,Sour cream,63409010.0,Guacamole,74402100.0,Salsa,14104010.0,"Cheese (cheddar, Swiss, etc.)",14109030.0,"Lowfat cheese (cheddar, Swiss, etc.)"
285,92101000,Coffee,91101010,White sugar (granulated or lump),12120100.0,Half & Half,12210200.0,Coffee Mate creamer (liquid),12210250.0,Coffee Mate Lite creamer (liquid),11112210.0,1% milk (low fat)
286,92101900,Latte,91101010,White sugar (granulated or lump),12140000.0,"Whipped cream, sweetened",NaN,NaN,NaN,NaN,NaN,NaN
287,92161000,Cappuccino,91101010,White sugar (granulated or lump),12140000.0,"Whipped cream, sweetened",NaN,NaN,NaN,NaN,NaN,NaN
288,92302000,Tea (unsweetened),91101010,White sugar (granulated or lump),11111000.0,Whole milk,11112210.0,1% milk (low fat),11113000.0,Fat free milk (skim),61204010.0,Lemon juice


In [16]:
food_condiments = pd.read_excel("/home/terrence/Projects/PycharmProjects/MachineLeaningTut/data/myfoodapediadata/lu_Condiment_Food_Table.xlsx")
food_condiments.tail()

,survey_food_code,display_name,condiment_portion_size,condiment_portion_code,condiment_grains,condiment_whole_grains,condiment_vegetables,condiment_dkgreen,condiment_orange,condiment_starchy_vegetables,...,condiment_milk,condiment_meat,condiment_soy,condiment_drybeans_peas,condiment_oils,condiment_solid_fats,condiment_added_sugars,condiment_alcohol,condiment_calories,condiment_saturated_fats
58,91300010,Syrup (regular),1 Tbsp,21000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00444,0.0,49.47393,0.0,53.00,0.00400
59,91302010,Honey,1 Tbsp,21000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,63.84000,0.0,63.84,0.00000
60,91361010,Sweet and sour sauce,1 Tbsp,21000,0.0657,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,10.09130,0.0,14.10,0.00015
61,91401000,Jelly (all flavors),1 Tbsp,21000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,27.46698,0.0,50.54,0.00190
62,91406500,All-fruit preserves,1 Tbsp,21000,0.0000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,36.60,0.01200
